In [1]:
import os

from os.path import  join, isdir
from plan import Plan
from action import Action
from utils import load_from_folder
from multiprocess import Pool
import random
from logging import exception
import re

random.seed(42)

In [2]:
save_dir = './new_plans/'
data_base_dir = '../datasets/'
domain = 'logistics'
results_dir = f"{save_dir}/{domain}/"   
source_dir = f"{join(data_base_dir, domain)}/optimal_plans/dictionaries_and_plans/" 
print('Domain dir:', source_dir)
os.makedirs(save_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

plans_to_process = 10 # number of plans to process
versions_per_plan = 6 # number of versions per each plan
number_of_goals = 4 # number of goals per each new plan
test = True # test will process only 3 plans

Domain dir: ../datasets/logistics/optimal_plans/dictionaries_and_plans/


In [3]:
plans = load_from_folder(source_dir,["plans"])[0]
print(f"Plans: {len(plans)}")

plans loaded from ../datasets/logistics/optimal_plans/dictionaries_and_plans/
Plans: 47769


In [31]:
def compute_recognizability(current_goal_state, goal_state_list):
    """
    Compute the difficulty of a plan.
    :param plan: The plan to compute the difficulty of.
    :param goal_state_list: The list of goal states.
    :return: The difficulty of the plan.
    """
    max_recognizability  = 1*len(current_goal_state)
    min_recognizability  = 1/(len(goal_state_list)) * len(current_goal_state)
    
    # print(f"Max recognizability : {max_recognizability}")
    # print(f"Min recognizability : {min_recognizability}")
    
    sum = 0
    #need to count how many times the current goal fluent is in the goal state list
    for current_goal_fluent in current_goal_state:
        count = 0
        for goal_state in goal_state_list:
            for goal_fluent in goal_state:       
                if current_goal_fluent==goal_fluent:
                    count += 1
        sum += 1/count
    
    #print(f"Unscaled recognizability: {sum}")
    
    #normalize the recognizability 
    recognizability = (sum-min_recognizability) / (max_recognizability-min_recognizability)
    
    return recognizability
                    

In [5]:
#compute difficulty testing
current_goal_state = [6, 5, 7, 9]
goal_state_list = [[1, 2, 3, 4], 
                   [1, 2, 3, 4], 
                   [1, 2, 3, 4], 
                   [1, 2, 3, 4], 
                   [1, 2, 3, 4]]
goal_state_list.append(current_goal_state)
rec = compute_recognizability(current_goal_state, goal_state_list)
print(f"recognizability : {rec}")

Unscaled recognizability: 4.0
recognizability : 1.0


In [6]:
 #todo gpt code check

import math

def approx_counts(num_of_fluents: int, num_of_states: int, r: float) -> list:
    """
    Return a list of n integer counts in [1..m] whose reciprocals sum
    approximately to S = n/m + r * n*(m-1)/m.

    Strategy:
      1. Compute the “continuous” count c = m / (1 + r*(m-1)).
      2. If c is (nearly) integer, just return [round(c)]*n.
      3. Otherwise let b = floor(c), B = b + 1, and solve for x:
            x*(1/b) + (n-x)*(1/B) = S
         => x = (n/B - S) / (1/B - 1/b)
      4. Round x to the nearest integer; assign x entries = b, (n-x) = B.
      5. Clip to [0..n] and, if desired, do a tiny local tweak
         (e.g. move 1 count up/down) to reduce the residual error.
    """
    # 1) target unscaled sum
    S = (num_of_fluents/num_of_states) + r * (num_of_fluents * (num_of_states-1) / num_of_states)

    # 2) continuous ideal count
    c = num_of_states / (1 + r*(num_of_states-1))
    c_round = round(c)
    # if it’s essentially integral, use it
    if abs(c_round - c) < 1e-6 or c_round in (1, num_of_states):
        return [c_round] * num_of_fluents

    # 3) floor / ceil
    b = math.floor(c)
    B = b + 1

    # solve x*(1/b) + (n-x)*(1/B) = S
    #   => x = (n/B - S) / (1/B - 1/b)
    denom = (1/B - 1/b)
    if abs(denom) < 1e-8:
        # degenerate; fallback to uniform
        return [c_round] * num_of_fluents

    x_real = (num_of_fluents/B - S) / denom
    x = int(round(x_real))

    # 4) clip and build
    x = max(0, min(num_of_fluents, x))
    counts = [b]*x + [B]*(num_of_fluents - x)

    # 5) (optional) tiny local corrections
    # compute residual error
    current_sum = sum(1/ci for ci in counts)
    # if we’re off by more than, say, 1/n, try one adjustment
    if abs(current_sum - S) > 1e-3:
        # if sum too small, we need to increase it ⇒ lower some ci by 1
        if current_sum < S:
            # find an index with ci > 1 and decrement it
            for i in range(num_of_fluents):
                if counts[i] > 1:
                    counts[i] -= 1
                    break
        else:
            # sum too big ⇒ decrement sum ⇒ increase some ci by 1
            for i in range(num_of_fluents):
                if counts[i] < num_of_states:
                    counts[i] += 1
                    break

    return counts


In [7]:
import random


def build_goal_list(current, goal_set_list_size, counts, fillers=['a','b','c','d','e']):
    num_of_goal_fluents = len(current)
    # initialize empty slots
    slots = [ [] for _ in range(goal_set_list_size) ]
    # for each fluent, choose which rows it goes in
    for fluent, c in zip(current, counts):
        rows = random.sample(range(goal_set_list_size), c)
        for r in rows:
            slots[r].append(fluent)
    # fill the rest with distractors
    all_distractors = ['a','b','c','d','e']  # pool of non-current fluents
    for r in range(goal_set_list_size):
        while len(slots[r]) < num_of_goal_fluents:
            slots[r].append(random.choice(all_distractors))
        random.shuffle(slots[r])
    return slots

# Example usage:
current = [6,5,7,9]
goal_set_list_size = 6
# counts chosen by solving sum(1/ci)=S (approximated)
counts = approx_counts(len(current), goal_set_list_size, 0.47)
print("Counts:", counts)
goal_list = build_goal_list(current, goal_set_list_size, counts)
print("Goal list:")
for i, g in enumerate(goal_list):
    print(f"Goal {i}: {g}")
rec = compute_recognizability(current, goal_list)
print(f"recognizability : {rec}")

Counts: [1, 2, 2, 2]
Goal list:
Goal 0: ['e', 'b', 'a', 5]
Goal 1: ['a', 'a', 9, 'a']
Goal 2: ['b', 'e', 7, 'e']
Goal 3: ['e', 'a', 'c', 'b']
Goal 4: ['c', 'a', 'b', 'b']
Goal 5: [6, 9, 5, 7]
Unscaled recognizability: 2.5
recognizability : 0.55


In [8]:
#if i want nine classes of recognizability i want unscaled
#divide a range 0.6666 to 4 in 9 classes
#4-0.6666 = 3.3334
#3.3334/9 = 0.3704
classes = []
for i in range(1, 10):
    classes.append(0.67 + i*0.3704)
print(f"Unscaled Classes: {[round(x,2) for x in classes]}")
print(f"Scaled Classes: {[round((x-0.67) / (4-0.67),2) for x in classes]}")




Unscaled Classes: [1.04, 1.41, 1.78, 2.15, 2.52, 2.89, 3.26, 3.63, 4.0]
Scaled Classes: [0.11, 0.22, 0.33, 0.44, 0.56, 0.67, 0.78, 0.89, 1.0]


In [50]:
def write_and_save_plan_and_versions(plan, goal_state_list, obj_set_dict={}):
    for i, goal_state in enumerate(goal_state_list):
        
        
        #extract plan name with regex
        name = re.search(r"(p\d+)(?=\.)", plan.plan_name).group(1)
        
        #definition
        new_problem = ""
        if i==0:
            new_problem += f";;(;metadata (recognizability:{round(compute_recognizability(goal_state_list[0], goal_state_list),2)})\n"
        
        new_problem += f"(define (problem {domain}_{name}_{i})\n(:domain {domain})\n(:objects\n\t"
    
        
        #objects in a dict format, {type: obj_set}
        for type, obj_set in obj_set_dict.items():
            if len(obj_set) > 0:
                for obj in obj_set:
                    new_problem += f"{obj} "
                new_problem += f"- {type}\n\t"
        new_problem += f")\n"
        
        #initial state
        new_problem += f"(:init\n"
        for fluent in plan.initial_state:
            new_problem += f"\t{fluent}\n"
        new_problem += f")\n"
        
        #goal state
        new_problem += f"(:goal (and\n"
        for goal in goal_state:
            new_problem += f"\t{goal}\n"
        new_problem += f"))\n)"
        #print(new_problem + "\n\n")
        
        #save the new problem in a file
        #naming convention is {current plan name_version number.pddl}, _0 is the original plan
        new_problem_dir = f"{results_dir}/{name}/"
        os.makedirs(new_problem_dir, exist_ok=True)
        new_problem_file = f"{new_problem_dir}/{name}_{i}.pddl"
        with open(new_problem_file, "w") as f:
            f.write(new_problem)

In [ ]:
def check_if_fluent_is_usable(fluent_to_add, goal_state):
    for fluent in goal_state:
        if fluent == fluent_to_add:
            return False
    return True

def adapt_goal_state_list_to_recognizability(base_goal_state, goal_state_list, recognizability=[0.1, 0.2]):
    
    goal_state_list = [list(x) for x in goal_state_list]
    
    all_goal_fluents = []
    for goal_state in goal_state_list:
        all_goal_fluents.append(goal_state)
        
    non_base_goal_fluents = []
    for fluent in all_goal_fluents:
        if fluent not in base_goal_state:
            non_base_goal_fluents.append(fluent)
    
    running_recognizability = compute_recognizability(base_goal_state, [base_goal_state] + goal_state_list)
    print(f"Starting recognizability: {running_recognizability}")

    while running_recognizability < recognizability[0] or running_recognizability > recognizability[1]:
        running_recognizability = compute_recognizability(base_goal_state, [base_goal_state] + goal_state_list)
        print(f"Running recognizability: {running_recognizability}")
        #shuffle the goal state list to improve randomness
        random.shuffle(goal_state_list)
        if running_recognizability > recognizability[1]:
            #find a goal state in goal_state_list that has a fluent that is not in the base goal state
            #swap it with one from usable_base_goal_fluents
            for goal_state in goal_state_list:
                
                #builds list of fluents that are in the base goal state but not in this goal state
                usable_base_goal_fluents = []
                for fluent in base_goal_state:  
                    if check_if_fluent_is_usable(fluent, goal_state):
                        usable_base_goal_fluents.append(fluent)
                                    
                if len(usable_base_goal_fluents) > 0:
                    #swap a fluent from the goal state with one from the base goal state
                    fluent_to_swap = random.choice(usable_base_goal_fluents)
                    
                    candidates_list = []
                    for fluent in goal_state:
                        if fluent not in base_goal_state:
                            candidates_list.append(fluent)
                    if len(candidates_list) == 0:
                        #if there are no candidates to swap, break
                        break
                    random_fluent = random.choice(candidates_list)
                    goal_state[goal_state.index(random_fluent)] = fluent_to_swap
                    
                    break
                
        elif running_recognizability < recognizability[0]:
            #choose a goal state that has a fluent from base goal and swap it with a random one
            for goal_state in goal_state_list:
                #builds list of fluents that are in the base goal state and also in this goal state
                present_base_goal_fluents = []
                for fluent in base_goal_state:
                    if not check_if_fluent_is_usable(fluent, goal_state):
                        present_base_goal_fluents.append(fluent)
                        break
                                    
                if len(present_base_goal_fluents) > 0:
                    #swap a common fluent with a random one
                    candidates_list = []
                    for fluent in non_base_goal_fluents:
                        if check_if_fluent_is_usable(fluent, goal_state):
                            candidates_list.append(fluent)
            
                    random_fluent = random.choice(candidates_list)
                    
                    fluent_to_swap = random.choice(present_base_goal_fluents)
                    goal_state[goal_state.index(present_base_goal_fluents)] = fluent_to_swap
                    
                    break
            
    return base_goal_state, goal_state_list

In [15]:
test= [["a"],["b"],["c"],["d"],["e"]]
test2 = ["f"]
merged = [test2] + test 

print(merged)

[['f'], ['a'], ['b'], ['c'], ['d'], ['e']]


In [ ]:
#process the objects in the plan then create the variations
count = 0
for plan in plans:
    if test:
        if count >= 3:
            break
    elif count > plans_to_process:
        break
    
    #begin plan processing
    
    all_obj_set = set()
    package_for_goal_set = set()
    pos_for_goal_set = set()
    
    #* find all objects in the initial state and actions
    for line in plan.initial_state:
        for obj in line.split(" ")[1:]:
            all_obj_set.add(obj)
    for action in plan.actions:
        for fluent in action.positiveEffects:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
        for fluent in action.negativeEffects:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
        for fluent in action.precondition:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
    
    #split the objects in their types
    pos_set = set()
    apn_set = set()
    cit_set = set()
    apt_set = set()
    tru_set = set()
    pack_set = set()
    obj_set_dict = {}
    for obj in all_obj_set:
        if obj.startswith("pos"):
            pos_set.add(obj)
            pos_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("obj"):
            pack_set.add(obj)
            package_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("apn"):
            apn_set.add(obj)
        elif obj.startswith("cit"):
            cit_set.add(obj)
        elif obj.startswith("tru"):
            tru_set.add(obj)
        elif obj.startswith("apt"):
            apt_set.add(obj)
    
    #useful for printing the plan
    if len(pos_set) > 0:
        obj_set_dict["location"] = pos_set
    if len(apn_set) > 0:
        obj_set_dict["airplane"] = apn_set
    if len(cit_set) > 0:
        obj_set_dict["city"] = cit_set
    if len(apt_set) > 0:
        obj_set_dict["airport"] = apt_set
    if len(tru_set) > 0:
        obj_set_dict["truck"] = tru_set
    if len(pack_set) > 0:
        obj_set_dict["package"] = pack_set
    
    #raise an exception if number of goals > number of packages or positions
    if number_of_goals > len(package_for_goal_set):
        raise exception(f"Number of goals {number_of_goals} is greater than the number of objects {len(package_for_goal_set)}")

    if number_of_goals > len(pos_for_goal_set):
        raise exception(f"Number of goals {number_of_goals} is greater than the number of positions {len(pos_for_goal_set)}")

    
    goals_state_list = []
    #* now we start creating the new versions of the problem from the current one
    for i in range(0, versions_per_plan+2):
        
        if i==0:
            goal_state_list.append(plan.goals[:number_of_goals]) # we keep the original goal state
            continue
            # we keep the original plan at index 0
        
        #working copy of the sets as we have to prevent a package being in different positions
        package_for_goal_set_copy = package_for_goal_set.copy()
        
        #? can two packages be in the same position?
        #? for example |at obj1 pos1| e |at obj2 pos1| 
        #? for now we assume no
        pos_for_goal_set_copy = pos_for_goal_set.copy()
        
        #create a set of goals of the type [at obj pos] 
        #by picking a random package and a random position from the sets
        #todo how to not have same goal_sets in two different versions? do we care about it enough?
        goal_set= set()    
        for k in range(0, number_of_goals):
            random_package = random.choice(list(package_for_goal_set_copy))
            #print(f"Random package: {random_package}")
            package_for_goal_set_copy.remove(random_package) # so we don't have conflicting goals
            random_pos = random.choice(list(pos_for_goal_set_copy))
            #print(f"Random position: {random_pos}")
            #? can two packages be in the same position?
            #? for example |at obj1 pos1| e |at obj2 pos1|
            #pos_for_goal_set_copy.remove(random_pos)
            goal_set.add(f"at {random_package} {random_pos}") 
        #print(f"Goal set: {goal_set}")
        
        #now we build the new pddl file with these goals
        goals_state_list.append(goal_set)
        
        
    
    
    base_goal_state, goals_state_list=adapt_goal_state_list_to_recognizability(base_goal_state=list(goals_state_list[0]), goal_state_list=list(goals_state_list[1:]), recognizability=[0.1, 0.2])
    print(f"Goals state list before: {goals_state_list}") 
    goals_state_list = [base_goal_state] + goals_state_list
    print(f"Goals state list after: {goals_state_list}")
    print(len(goals_state_list))
    write_and_save_plan_and_versions(plan=plan, goal_state_list=goals_state_list, obj_set_dict=obj_set_dict) #todo to be adapted to write plan and its versions

    count = count + 1

Starting recognizability: 0.85
Running recognizability: 0.85
Running recognizability: 0.7000000000000001
Running recognizability: 0.6499999999999999
Running recognizability: 0.625
Running recognizability: 0.47500000000000003
Running recognizability: 0.325
Running recognizability: 0.30999999999999994
Running recognizability: 0.25999999999999995
Running recognizability: 0.20999999999999994
Goals state list before: [['at obj66 pos66', 'at obj00 pos66', 'at obj11 pos66', 'at obj55 pos66'], ['at obj66 pos66', 'at obj66 pos55', 'at obj44 pos11', 'at obj55 pos66'], ['at obj66 pos23', 'at obj11 pos66', 'at obj23 pos44', 'at obj44 pos22'], ['at obj66 pos13', 'at obj55 pos66', 'at obj33 pos22', 'at obj11 pos66'], ['at obj55 pos66', 'at obj13 pos44', 'at obj44 pos13', 'at obj55 pos23']]
Goals state list after: [['at obj00 pos66', 'at obj55 pos66', 'at obj66 pos66', 'at obj11 pos66'], ['at obj66 pos66', 'at obj00 pos66', 'at obj11 pos66', 'at obj55 pos66'], ['at obj66 pos66', 'at obj66 pos55', 'at